# Playbooks

Describe in more details playbooks.

Show some example playbooks with:

 - user and skel
 - install packages
 - enable service #not on containers
 - lineinfile
 - curl
 - file
 - copy
 - fetch
 
Creating small reports.

More fun with:

  - iterations
  - ignore_errors
  - when
  - with_fileglob and remote_fileglob ;)

In [1]:
cd exercise-06

/notebooks/exercise-06


## Gathering facts

playbooks are groups of tasks we can run with ansible.

A playbook starts with an hosts mark specifying the hosts to run the playbook to.

```
---
- hosts: web
  tasks: 
  ...
```

When run, a playbook gathers facts about hosts (remember the setup module?).

Gathering facts may be time-consuming, so you can disabe that with

```
- hosts: web
  gather_facts: no
  tasks:
  ...
```



In [2]:
!cat debug.yml


- hosts: localhost
  name: those tasks are run on localhost only
  tasks:
  - name: This is the debug module.
    debug: msg="A string"
    
  - name: The debug module can print a variable too
    debug: var=ansible_hostname

  - name: We can iterate too
    debug: msg="{{item}}"
    with_items: "{{ansible_system_capabilities}}"

  - name: and inspect/format variables
    debug: msg="The device is {{item.device}}"
    with_items: >
      {{ansible_mounts}}



In [3]:
!ansible-playbook debug.yml


PLAY [those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [This is the debug module.] ***********************************************
ok: [localhost] => {
    "msg": "A string"
}

TASK [The debug module can print a variable too] *******************************
ok: [localhost] => {
    "ansible_hostname": "sysadminpy"
}

TASK [We can iterate too] ******************************************************
ok: [localhost] => (item=cap_chown) => {
    "item": "cap_chown", 
    "msg": "cap_chown"
}
ok: [localhost] => (item=cap_dac_override) => {
    "item": "cap_dac_override", 
    "msg": "cap_dac_override"
}
ok: [localhost] => (item=cap_fowner) => {
    "item": "cap_fowner", 
    "msg": "cap_fowner"
}
ok: [localhost] => (item=cap_fsetid) => {
    "item": "cap_fsetid", 
    "msg": "cap_fsetid"
}
ok: [localhost] => (item=cap_kill) => {
    "item": "cap_kill", 
    "msg"

## Exercise: 

[modify debug.yml](/edit/notebooks/exercise-06/debug.yml) to disable fact gathering and use the following cell to test it 

In [4]:
!ansible-playbook debug.yml


PLAY [those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [This is the debug module.] ***********************************************
ok: [localhost] => {
    "msg": "A string"
}

TASK [The debug module can print a variable too] *******************************
ok: [localhost] => {
    "ansible_hostname": "sysadminpy"
}

TASK [We can iterate too] ******************************************************
ok: [localhost] => (item=cap_chown) => {
    "item": "cap_chown", 
    "msg": "cap_chown"
}
ok: [localhost] => (item=cap_dac_override) => {
    "item": "cap_dac_override", 
    "msg": "cap_dac_override"
}
ok: [localhost] => (item=cap_fowner) => {
    "item": "cap_fowner", 
    "msg": "cap_fowner"
}
ok: [localhost] => (item=cap_fsetid) => {
    "item": "cap_fsetid", 
    "msg": "cap_fsetid"
}
ok: [localhost] => (item=cap_kill) => {
    "item": "cap_kill", 
    "msg"

### Hints

 - use ansible_facts as possible instead of gathering facts with uname & co
 - test and template your iterations statically instead of continuosly gathering facts
 

In [5]:
!cat process-facts.yml



---
- hosts: localhost
  gather_facts: no
  vars:
  - ansible_mounts:
    - {device: '/dev/mapper/docker-253:2-537280731-eb92264366fcbaf248d3d5a93bbaab17eec130b43ff94e44e11abab77f5f2038',
      fstype: xfs, mount: /, options: 'rw,context="system_u:object_r:svirt_sandbox_file_t:s0:c230,c811",relatime,nouuid,attr2,inode64,logbsize=64k,sunit=128,swidth=128,noquota,bind',
      size_available: 107076997120, size_total: 107320705024, uuid: N/A}
    - {device: /dev/mapper/vg0-home, fstype: xfs, mount: /code, options: 'rw,seclabel,relatime,attr2,inode64,noquota,bind',
      size_available: 18586177536, size_total: 214643507200, uuid: N/A}
    
  tasks:
  - name: Test complex entries processing without gathering facts.
    debug: 
      msg: >
        {{item.device}} {{ (100 * item.size_available/item.size_total) | int }}%
    with_items: >
      {{ansible_mounts}}



In [6]:
!ansible-playbook process-facts.yml | grep msg

    "msg": "/dev/mapper/docker-253:2-537280731-eb92264366fcbaf248d3d5a93bbaab17eec130b43ff94e44e11abab77f5f2038 99%\n"
    "msg": "/dev/mapper/vg0-home 8%\n"


## Packages, User and Files

Creating user and installing packages is easy

```
  - name: Install apache
    apt: item="{{item}}" state=present
    with_items:
    - apache2
    - curl
    
  - name: Remove wget
    apt: item=wget state=absent
```

In [8]:
!cat package-user.yml


- hosts: web
  tasks:
  - name: Install apache and other packages eventually iterating
    apt: name="{{item}}"
    with_items:
    - apache2
    - curl

  - name: Remove wget
    apt: name=wget state=absent

  - name: Create user
    user: 
      name: foo
      groups: adm
      append: yes
      shell: /bin/bash
      skeleton: /root   # duplicate /root/ content for the foo user

  - name: >
      Copy a file to ~foo expandig tilde.
      Do not forget quotes!
    copy:
      src: package-user.yml
      dest: "{{ '~foo' | expanduser }}"

  - name: >
      We can fetch files too (eg. logs):
      - files are dispatched to different directories.
      - we can avoid enforcing checksum as we're messing with 
        containers.
      
    fetch:
      src: /etc/passwd
      dest: /tmp/fetched
      validate_checksum: no


In [9]:
!ansible-playbook package-user.yml



PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [ansible101_web_1]

TASK [Install apache and other packages eventually iterating] ******************
ok: [ansible101_web_1] => (item=[u'apache2', u'curl'])

TASK [Remove wget] *************************************************************
ok: [ansible101_web_1]

TASK [Create user] *************************************************************
ok: [ansible101_web_1]

TASK [Copy a file to ~foo expandig tilde. Do not forget quotes!] ***************
ok: [ansible101_web_1]

TASK [We can fetch files too (eg. logs): - files are dispatched to different directories. - we can avoid enforcing checksum as we're messing with 
  containers.] ***
changed: [ansible101_web_1]

PLAY RECAP *********************************************************************
ansible101_web_1           : ok=6    changed=1    unreachable=0    failed=0   



In [12]:
!tree /tmp/fetched/

/tmp/fetched/
└── ansible101_web_1
    ├── etc
    └── var
        └── log

4 directories, 0 files


## Files and Directories

Creating files and directories. 

lineinfile / blockinfile

with_fileglob

In [11]:
!ansible-playbook -i inventory files-content.yml 



PLAY [A playbook can be made of multiple stanzas. They'll be pipelined.] *******

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Delete previous template] ************************************************
changed: [localhost]

TASK [We're now just showing how to create a file containing a block of lines] ***
changed: [localhost]

TASK [Register command output in a variable] ***********************************
changed: [localhost]

TASK [Show output] *************************************************************
ok: [localhost] => {
    "o_cat.stdout": "False\n<html> <body> System installed by sysadminpy on {{ ansible_hostname }} </body> </html>\nFalse"
}

PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [ansible101_web_1]

TASK [Install apache and other packages eventually iterating] ******************
ok: [ansible

## shell module reloaded

Ansible can be used to reproduce issues and gather command output. 

Though the standard workflow can be done registering output in a temporary variable


In [14]:
!cat shell-output-01.yml

# 
- hosts: localhost
  tasks:
  - name: Run multiple commands via /bin/sh (no bashisms)
    shell: |
      ls -latr /etc/host*
    ignore_errors: yes
    register: o
    
  - name: Show output in a single block...
    debug: var=o.stdout

  - name: ... or one per line
    debug: var=item
    with_items: >
      {{ o.stdout_lines }}



In [15]:
!ansible-playbook shell-output-01.yml


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Run multiple commands via /bin/sh (no bashisms)] *************************
changed: [localhost]

TASK [Show output in a single block...] ****************************************
ok: [localhost] => {
    "o.stdout": "-rw-r--r--. 1 root root   9 Aug  7  2006 /etc/host.conf\n-rw-r--r--. 1 root root 172 Jun 29 09:31 /etc/hosts\n-rw-r--r--. 1 root root  11 Jun 29 09:31 /etc/hostname"
}

TASK [... or one per line] *****************************************************
ok: [localhost] => (item=-rw-r--r--. 1 root root   9 Aug  7  2006 /etc/host.conf) => {
    "item": "-rw-r--r--. 1 root root   9 Aug  7  2006 /etc/host.conf"
}
ok: [localhost] => (item=-rw-r--r--. 1 root root 172 Jun 29 09:31 /etc/hosts) => {
    "item": "-rw-r--r--. 1 root root 172 Jun 29 09:31 /etc/hosts"
}
ok: [localhost] => (item=-rw-r--r--. 

This approach has its limits.

### Exercise:

  - what happens if you processa long pipeline with  [shell-output-01.yml](/edit/notebooks/exercise-07/shell-output-01.yml) ? 
  
Exercise: modify [shell-output-01.yml](/edit/notebooks/exercise-07/shell-output-01.yml) so that:

  - every command stdout/stderr is redirected to a given file
  - before and after every command output print  a header and a footer (eg. the expected output is like
  
```
--- START COMMAND: cat /etc/resolv.conf --
nameserver 172.17.0.1
-- END COMMAND: cat /etc/resolv.conf
```

HINTS:

  - use shell redirection instead of `register`
  - use with_items to process many different commands 
  

## Templates

Creating files from templates.

include template files, enforcing policies

tagging

